In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import StepLR
from utils import CustomDataset, train, train_with_int_input

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Doc2idxs as input

In [ ]:
if torch.cuda.is_available(): # in my case it means running in colab
    from google.colab import drive
    drive.mount('/content/drive')
    from pathlib import Path
    sources_path = Path("/content/drive/MyDrive/Data")
    X_train = np.load(str(sources_path / 'doc2idxs_X_train.npy'))
    y_train = np.load(str(sources_path / 'y_train.npy'))
    X_test = np.load(str(sources_path / 'doc2idxs_X_test.npy'))
    y_test = np.load(str(sources_path / 'y_test.npy'))

    saving_path=str(sources_path / 'GRU_doc2idxs.pth')
else:
    X_train = np.load('Data/doc2idxs_X_train.npy')
    y_train = np.load('Data/y_train.npy')
    X_test = np.load('Data/doc2idxs_X_test.npy')
    y_test = np.load('Data/y_test.npy')

    saving_path='Trained_models/GRU_doc2idxs.pth'

train_df = CustomDataset(X_train, y_train)
test_df = CustomDataset(X_test, y_test)

In [ ]:
class GRU_doc2idxs(nn.Module):
    def __init__(self,voc_size, embedding_size, padding_idx, hidden_state_size, n_rec_layers, output_size):
        super().__init__()
        self.n_rec_layers = n_rec_layers
        self.hidden_state_size = hidden_state_size

        self.embedding = nn.Embedding(voc_size, embedding_size, padding_idx)
        self.gru = nn.GRU(embedding_size, hidden_state_size, n_rec_layers, batch_first=True, dropout=0.3)
        self.bn = nn.BatchNorm1d(hidden_state_size)
        self.fc = nn.Linear(hidden_state_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, X):
        h_0 = torch.zeros(self.n_rec_layers, X.size(0), self.hidden_state_size).to(device)

        X, _ = self.gru(X, h_0)
        X = X[:, -1:, :].flatten(start_dim=1)
        X = self.bn(X)
        X = F.relu(X)
        X = self.fc(X)
        X = self.logsoftmax(X)

        return X

In [ ]:
gru = GRU_doc2idxs(
    voc_size=36879, # from Pre-processing.ipynb
    embedding_size=512,
    padding_idx=0, # from Pre-processing.ipynb
    hidden_state_size=512,
    n_rec_layers=1,
    output_size=5
).to(device)

optimizer = Adam(gru.parameters(), lr=1e-4),

train_with_int_input(gru,
      optimizer=optimizer,
      train_loader=DataLoader(train_df, batch_size=256, shuffle=True),
      test_loader=DataLoader(test_df, batch_size=256, shuffle=False),
      epochs=45,
      saving_path=saving_path,
      scheduler=StepLR(optimizer, step_size=1, gamma=0.99))

In [ ]:
with torch.no_grad():
    y_pred = torch.argmax(gru(torch.from_numpy(X_test).to(device)), dim=1).cpu().numpy() - 2
    print(y_pred)
    print(classification_report(y_test, y_pred))

### Doc2matrix input

In [2]:
if torch.cuda.is_available(): # in my case it means running in colab
    from google.colab import drive
    drive.mount('/content/drive')
    from pathlib import Path
    sources_path = Path("/content/drive/MyDrive/Data")
    X_train = np.load(str(sources_path / 'doc2matrix_X_train.npy'))
    y_train = np.load(str(sources_path / 'y_train.npy'))
    X_test = np.load(str(sources_path / 'doc2matrix_X_test.npy'))
    y_test = np.load(str(sources_path / 'y_test.npy'))

    saving_path=str(sources_path / 'GRU_doc2matrix.pth')
else:
    X_train = np.load('Data/doc2matrix_X_train.npy')
    y_train = np.load('Data/y_train.npy')
    X_test = np.load('Data/doc2matrix_X_test.npy')
    y_test = np.load('Data/y_test.npy')

    saving_path='Trained_models/GRU_doc2matrix.pth'

train_df = CustomDataset(X_train, y_train)
test_df = CustomDataset(X_test, y_test)

In [23]:
class GRU_doc2matrix(nn.Module):
    def __init__(self, hidden_state_size, n_rec_layers, input_size, output_size):
        super().__init__()
        self.n_rec_layers = n_rec_layers
        self.hidden_state_size = hidden_state_size

        self.gru = nn.GRU(input_size, hidden_state_size, n_rec_layers, batch_first=True, dropout=0.3)
        self.bn = nn.BatchNorm1d(hidden_state_size)
        self.fc = nn.Linear(hidden_state_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, X):
        h_0 = torch.zeros(self.n_rec_layers, X.size(0), self.hidden_state_size).to(device)

        X, _ = self.gru(X, h_0)
        X = X[:, -1:, :].flatten(start_dim=1)
        X = self.bn(X)
        X = F.relu(X)
        X = self.fc(X)
        X = self.logsoftmax(X)

        return X

In [24]:
gru = GRU_doc2matrix(
    hidden_state_size=512,
    n_rec_layers=2,
    input_size=200,
    output_size=5
).to(device)

optimizer = Adam(gru.parameters(), lr=1e-4)

train(gru,
      optimizer,
      train_loader=DataLoader(train_df, batch_size=256, shuffle=True),
      test_loader=DataLoader(test_df, batch_size=256, shuffle=False),
      epochs=45,
      saving_path=saving_path,
      scheduler=StepLR(optimizer, step_size=1, gamma=0.99))

  0%|          | 0/45 [00:09<?, ?it/s]

torch.Size([1024, 10, 512])


RuntimeError: running_mean should contain 10 elements not 512

In [23]:
with torch.no_grad():
    y_pred = torch.argmax(gru(torch.from_numpy(X_test).to(device)), dim=1).cpu().numpy() - 2
    print(y_pred)
    print(classification_report(y_test, y_pred))

[ 0  0  1 ... -1 -1  0]
              precision    recall  f1-score   support

          -2       0.49      0.73      0.58       473
          -1       0.64      0.69      0.67      2746
           0       0.82      0.65      0.72      4174
           1       0.46      0.75      0.57       522
           2       0.45      0.79      0.58       105

    accuracy                           0.68      8020
   macro avg       0.57      0.72      0.62      8020
weighted avg       0.71      0.68      0.68      8020

